# RL Q-Learning Taxi example

Importing the relevant libraries

In [3]:
import gym
import numpy as np
import random
from IPython.display import clear_output
from time import sleep

### Environment

### Action space = 6 possible Actions  
1 south  
2 north  
3 east  
4 west  
5 pickup  
6 dropoff  

In [7]:
env = gym.make('Taxi-v2').env

#Reset the Environment to random state
env.reset()
env.render()

print("Action Space {}".format(env.action_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)


### State space = 500 States  
4 pickup locations  
5x5 grid  
1 additional passenger (x5)

In [8]:
print("State Space {}".format(env.observation_space))

State Space Discrete(500)


### Encoding a State

In [11]:
# (taxi row, taxi column, passenger index, destination index)
state = env.encode(3, 1, 2, 0)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Reward table

{action: [(probability, nextstate, reward, done)]}  
0-5 corresponds to the actions (south, north, east, west, pickup, dropoff)  
Probability is always 1.0  
nextstate is state of action at this index of the dict  
movements have a -1 reward  
pickup/dropoff have -10, at right location +20 reward

In [36]:
env.P[228]

{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

### Implementing Q-learning

Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.  
In our Taxi environment, we have the reward table, P, that the agent will learn from. It takes an action in the current state, recieves the reward for this action and then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.

Better Q-values imply better chances of getting greater rewards.

In [16]:
#Initializing the Q-table to a 500x6 Matrix of 0
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [17]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:

\begin{equation*}
Q(state, action) = \small{(1-\alpha) Q (state,action) + \alpha(reward + \gamma maxQ(nextState, allActions)}
\end{equation*}

$\alpha$ is the learning rate (between 0 and 1), the extent to which the Q-values are being updated in every iteration.

$\gamma$ is the discount factor (between 0 and 1), determines how important the future rewards are

In [18]:
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

We are assigning (←), or updating, the Q-value of the agent's current state and action by first taking a weight (1−α) of the old Q-value, then adding the learned value. 

The learned value is a combination of the reward for taking the current action in the current state, and the discounted maximum reward from the next state we will be in once we take the current action.

### Exploiting learned values

After enough random exploration of actions, the Q-values converge, serving as actoun-value function, where the agent can exploit the most optimal action from a given state.

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, with the parameter ϵ 

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [20]:
# Training the agent

# Plotting metrics
all_epochs = []
all_penalties = []
frames = [] # for animation
total_reward_list = []
avg_reward = [0]
total_reward, avg_reward_tot, avg_reward_episodes, total_epochs, episodes = 0,0,0,0,0

# Training loop
for i in range(1, 100001):
    state = env.reset()
    
    # Setting initial values
    epochs, penalties, reward = 0,0,0
    done = False

    # Decide wether to pick a random action or exploit the already computed Q-values (comparing to epsilon)
    while not done:
        if random.uniform(0,1) < epsilon:
            # Explore the action space (Number between 1 and 6)
            action = env.action_space.sample()
        else:
            # Exploit learned values (Maximum in Q_table of current state)
            action = np.argmax(q_table[state]) 
        
        #Execute the chosen action to obtain next state and reward
        next_state, reward, done, info = env.step(action)
        
        # Update the Q Value to the new Q Value (of next state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        # Q-learning equation:
        # (1-alpha) * Q(state, action) + alpha(reward_t+1 + gamma * maxQ(state_t+1, action))
        new_value = (1-alpha) * old_value + alpha * (reward + gamma * next_max)
        
        q_table[state, action]= new_value
        
        # Update Penalties and epochs
        if reward == -10:
            penalties +=1

        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward,
            'avg_reward_tot': avg_reward_tot,
            'avg_reward': avg_reward_episodes
            }
        )
        
        # Update state and reward
        state = next_state
        epochs +=1
        total_epochs+=1
        total_reward+=reward
        total_reward_list.append(reward)
        avg_reward_tot = total_reward/total_epochs
        avg_reward_episodes = avg_reward[episodes]

    # Clear output after 100 episodes
    if i%100 ==0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        episodes +=1
        avg_reward.append(sum(total_reward_list)/len(total_reward_list))
        total_reward_list = []

Episode: 100000


### Evaluating performance

In [22]:
# Print frames function for Q-Learning vizualisation      
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Avg. reward (100 episodes): {frame['avg_reward']}")
        print(f"Avg. total reward: {frame['avg_reward_tot']}")
        sleep(.1)

Agents performance on the first few runs

In [37]:
print_frames(frames[0:100])
print("Training finished.\n")

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 100
State: 133
Action: 2
Reward: -1
Avg. reward (100 episodes): 0
Avg. total reward: -2.5454545454545454
Training finished.



Agents Performance after 50k episodes

In [38]:
print_frames(frames[round(len(frames)/2):round(len(frames)/2)+100])
print("Training finished.\n")

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 100
State: 101
Action: 1
Reward: -1
Avg. reward (100 episodes): 0.22505307855626328
Avg. total reward: 0.00892458487298763
Training finished.



Agents Performance at 100k episodes

In [39]:
# Console output (Last 1000 training runs)
print_frames(frames[len(frames)-200:len(frames)])
print("Training finished.\n")

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 200
State: 97
Action: 5
Reward: 20
Avg. reward (100 episodes): 0.24264178033022255
Avg. total reward: 0.11515231823180994
Training finished.



### Q Values at the illustrations state
1 south  
2 north  
3 east  
4 west  
5 pickup  
6 dropoff 

In [40]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
env.render()
env.P[328]

q_table[328]

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


array([-2.30435943, -1.97092096, -2.30096055, -2.22305168, -9.79587531,
       -9.6383713 ])